In [1]:
import numpy as np
import pandas as pd
import nupack as nu
import networkx as nx 
import matplotlib.pyplot as plt

from hdna.complex import Complex
from hdna.strand import Strand
from hdna.model import Model
from hdna.chamber import Chamber
from hdna.kinetwork import Kinetwork
from hdna.simulator import Simulator, Options

from pyvis.network import Network

## About Julia-Python interface

In [2]:
import juliacall


jl = juliacall.newmodule('Dude')

jl.seval('using BioSimulator')
jl.seval('using TickTock')
jl.seval('using Base.Threads')

print(jl.nthreads())

jl.tick()
model = jl.Network("AT-all")  

initial = 2

model <= jl.Species("SS",int(initial))
model <= jl.Species("L1",0)
model <= jl.Species("L2",0)
model <= jl.Species("R1",0)
model <= jl.Species("R2",0)
model <= jl.Species("D",0)



kf_dl = 4e8
nonspecific_correction = 1/5

sliding_correction = 1/2
kf_sliding = 1e7 * sliding_correction



Na = 6.023e23

kf_ssl1 = kf_dl*nonspecific_correction
kb_ssl1 = 22.920792678077465 

kf_ssl2 = kf_dl*nonspecific_correction
kb_ssl2 = 0.007808225837362128

kf_ssr2 = kf_dl*nonspecific_correction
kb_ssr2 = 0.0626785751957368

kf_ssr1 = kf_dl*nonspecific_correction
kb_ssr1 = 183.99091641848813

kf_ssD = kf_dl*nonspecific_correction
kb_ssD = 6.902831840829526e-08

kf_l1l2 = kf_sliding
kb_l1l2 = 3406.6124793451313*sliding_correction

kf_l2D = kf_sliding
kb_l2D = 88.40461309148736*sliding_correction

kf_r1r2 = kf_sliding
kb_r1r2 = 3406.6124793451313*sliding_correction

kf_r2D = kf_sliding
kb_r2D = 11.01306438328072*sliding_correction




model <= jl.Reaction("ss-l1_f",  kf_ssl1, "SS + SS --> L1")
model <= jl.Reaction("ss-l1_b",  kb_ssl1, "L1 --> SS + SS")
model <= jl.Reaction("ss-l2_f",  kf_ssl2, "SS + SS --> L2")
model <= jl.Reaction("ss-l2_b",  kb_ssl2, "L2 --> SS + SS")
model <= jl.Reaction("ss-r2_f",  kf_ssr2, "SS + SS --> R2")
model <= jl.Reaction("ss-r2_b",  kb_ssr2, "R2 --> SS + SS")
model <= jl.Reaction("ss-r1_f",  kf_ssr1, "SS + SS --> R1")
model <= jl.Reaction("ss-r1_b",  kb_ssr1, "R1 --> SS + SS")
model <= jl.Reaction("ss-D_f",   kf_ssD, "SS + SS --> D")
model <= jl.Reaction("ss-D_b",   kb_ssD, "D --> SS + SS")
model <= jl.Reaction("l1-l2_f",  kf_l1l2, "L1 --> L2")
model <= jl.Reaction("l1-l2_b",  kb_l1l2, "L2 --> L1")
model <= jl.Reaction("l2-D_f",   kf_l2D, "L2 --> D")
model <= jl.Reaction("l2-D_b",   kb_l2D, "D --> L2")
model <= jl.Reaction("r1-r2_f",  kf_r1r2, "R1 --> R2")
model <= jl.Reaction("r1-r2_b",  kb_r1r2, "R2 --> R1")
model <= jl.Reaction("r2-D_f",   kf_r2D, "R2 --> D")
model <= jl.Reaction("r2-D_b",   kb_r2D, "D --> R2")

model <= jl.Species("oobbooAooddoo", 0)
model <= jl.Reaction("oobbooAooddoo", 0, "D --> oobbooAooddoo")



runtime = 1e-2
Nmonte = 50000
jl.tock()

jl.tick()
simresults = [jl.simulate(model, jl.Direct(),tfinal=runtime) for i in range(Nmonte)]
results = [{'c':jl.hcat(*simresults[i].u).__array__(), 't':simresults[i].t} for i in range(Nmonte)]
jl.tock()

1


[ Info:  started timer at: 2023-01-12T22:13:12.251
[ Info:            1.5377401s: 1 second, 537 milliseconds
[ Info:  started timer at: 2023-01-12T22:13:13.932
[ Info:            8.4730327s: 8 seconds, 473 milliseconds


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [4]:
#definitive routine for mfpis is detailed below

index = 2

results[index]['c']
results[index]['c'][-1]
results[index]['t']

fpi = np.argmax(results[index]['c'][-1] == 1)

fpi, results[index]['t'][fpi]

array([[2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0]])

array([0, 0, 0, 0, 0, 0])

6-element Vector{Float64}:
 0.0
 2.3840802963583688e-9
 2.6792317628218643e-9
 0.0037658392236347214
 0.003765893375974613
 0.01

(0, 0.0)

## About Chamber

In [1]:
import numpy as np
import pandas as pd
import nupack as nu
import networkx as nx 
import matplotlib.pyplot as plt

from hdna.complex import Complex
from hdna.strand import Strand
from hdna.model import Model, Geometry
from hdna.chamber import Chamber
from hdna.kinetwork import Kinetwork, Kinetics
from hdna.simulator import Simulator, Options #it slows down import because it has to wake julia up 

from pyvis.network import Network

In [3]:
model = Model('dna', '3D')
mincore = 3

a = Strand(model, 'AAAAAAAAAAA')
b = Strand(model, 'TTTTTTTTTTT')

kgraph = Kinetwork(model, a, b, mincore)
geometry = Geometry(180, 180)
kinetics = Kinetics(model, kgraph, geometry)
simulator = Simulator(model, kgraph, kinetics)
G = simulator.Graph

In [5]:
kin = Kinetics(model, kgraph)
type(kin.s1.length), type(kin.simplex_geometry['basepairdistance'])
kin.ss_strands_size()
kin.size1
kin.vanilla_diffusivities()
kin.D2

3.386589963937578e-18

In [12]:
a = list(nx.neighbors(simulator.Graph, simulator.kinet.chamber.singlestranded.structure))

a[1]
simulator.Graph.nodes[a[1]]['object'].G

-4.8833275148195945

In [25]:
SS = simulator.kinet.chamber.singlestranded.structure

sub = simulator.Graph.copy()
sub.remove_node(SS)
a,b,c = zip(*list(sub.edges.data()))
c[1]['kind']

'sliding'

In [3]:
simulator.tl(simulator.kinet.chamber.duplex.structure)

'bbbbbbbbbbbAddddddddddd'

In [4]:
G2 = simulator.Graph.copy()
G2.remove_node(simulator.kinet.chamber.singlestranded.structure)
print(*list(G2.edges.data()),sep='\n')

('(((........+)))........', '((((.......+)))).......', {'kind': 'sliding'})
('((((.......+)))).......', '(((((......+)))))......', {'kind': 'sliding'})
('(((((......+)))))......', '((((((.....+)))))).....', {'kind': 'sliding'})
('((((((.....+)))))).....', '(((((((....+)))))))....', {'kind': 'sliding'})
('(((((((....+)))))))....', '((((((((...+))))))))...', {'kind': 'sliding'})
('((((((((...+))))))))...', '(((((((((..+)))))))))..', {'kind': 'sliding'})
('(((((((((..+)))))))))..', '((((((((((.+)))))))))).', {'kind': 'sliding'})
('((((((((((.+)))))))))).', '(((((((((((+)))))))))))', {'kind': 'sliding-end'})
('.((((((((((+.))))))))))', '..(((((((((+..)))))))))', {'kind': 'sliding'})
('.((((((((((+.))))))))))', '(((((((((((+)))))))))))', {'kind': 'sliding-end'})
('..(((((((((+..)))))))))', '...((((((((+...))))))))', {'kind': 'sliding'})
('...((((((((+...))))))))', '....(((((((+....)))))))', {'kind': 'sliding'})
('....(((((((+....)))))))', '.....((((((+.....))))))', {'kind': 'sliding'})
('..

In [5]:
simulator.add_reactions()

In [ ]:
for i in simulator.add_reactions():
    print(i)
    print(G.nodes[i])

JuliaError: Base.Meta.ParseError("invalid identifier name \"...\"")
Stacktrace:
 [1] #parse#3
   @ ./meta.jl:237 [inlined]
 [2] parse(str::String; raise::Bool, depwarn::Bool)
   @ Base.Meta ./meta.jl:268
 [3] parse
   @ ./meta.jl:267 [inlined]
 [4] parse_reaction(formula::String)
   @ BioSimulator ~/.julia/packages/BioSimulator/RunkJ/src/interface/well-mixed/reaction.jl:88
 [5] BioSimulator.Reaction(name::String, rate::Int64, formula::String)
   @ BioSimulator ~/.julia/packages/BioSimulator/RunkJ/src/interface/well-mixed/reaction.jl:32
 [6] pyjlany_call(self::Type{BioSimulator.Reaction}, args_::PythonCall.Py, kwargs_::PythonCall.Py)
   @ PythonCall ~/.julia/packages/PythonCall/3GRYN/src/jlwrap/any.jl:31
 [7] _pyjl_callmethod(f::Any, self_::Ptr{PythonCall.C.PyObject}, args_::Ptr{PythonCall.C.PyObject}, nargs::Int64)
   @ PythonCall ~/.julia/packages/PythonCall/3GRYN/src/jlwrap/base.jl:69
 [8] _pyjl_callmethod(o::Ptr{PythonCall.C.PyObject}, args::Ptr{PythonCall.C.PyObject})
   @ PythonCa

In [ ]:
G1 = G.copy()
G1.remove_node('...........+...........')

In [ ]:
list(G1.edges.data())

[('(((........+)))........', '((((.......+)))).......', {'kind': 'sliding'}),
 ('((((.......+)))).......', '(((((......+)))))......', {'kind': 'sliding'}),
 ('(((((......+)))))......', '((((((.....+)))))).....', {'kind': 'sliding'}),
 ('((((((.....+)))))).....', '(((((((....+)))))))....', {'kind': 'sliding'}),
 ('(((((((....+)))))))....', '((((((((...+))))))))...', {'kind': 'sliding'}),
 ('((((((((...+))))))))...', '(((((((((..+)))))))))..', {'kind': 'sliding'}),
 ('(((((((((..+)))))))))..', '((((((((((.+)))))))))).', {'kind': 'sliding'}),
 ('((((((((((.+)))))))))).',
  '(((((((((((+)))))))))))',
  {'kind': 'sliding-end'}),
 ('.((((((((((+.))))))))))', '..(((((((((+..)))))))))', {'kind': 'sliding'}),
 ('.((((((((((+.))))))))))',
  '(((((((((((+)))))))))))',
  {'kind': 'sliding-end'}),
 ('..(((((((((+..)))))))))', '...((((((((+...))))))))', {'kind': 'sliding'}),
 ('...((((((((+...))))))))', '....(((((((+....)))))))', {'kind': 'sliding'}),
 ('....(((((((+....)))))))', '.....((((((+.....)

In [ ]:
for n1, n2, data in list(G1.edges.data()):
    print(n1, n2, data)
    print('\n')

(((........+)))........ ((((.......+))))....... {'kind': 'sliding'}


((((.......+))))....... (((((......+)))))...... {'kind': 'sliding'}


(((((......+)))))...... ((((((.....+))))))..... {'kind': 'sliding'}


((((((.....+))))))..... (((((((....+))))))).... {'kind': 'sliding'}


(((((((....+))))))).... ((((((((...+))))))))... {'kind': 'sliding'}


((((((((...+))))))))... (((((((((..+))))))))).. {'kind': 'sliding'}


(((((((((..+))))))))).. ((((((((((.+)))))))))). {'kind': 'sliding'}


((((((((((.+)))))))))). (((((((((((+))))))))))) {'kind': 'sliding-end'}


.((((((((((+.)))))))))) ..(((((((((+..))))))))) {'kind': 'sliding'}


.((((((((((+.)))))))))) (((((((((((+))))))))))) {'kind': 'sliding-end'}


..(((((((((+..))))))))) ...((((((((+...)))))))) {'kind': 'sliding'}


...((((((((+...)))))))) ....(((((((+....))))))) {'kind': 'sliding'}


....(((((((+....))))))) .....((((((+.....)))))) {'kind': 'sliding'}


.....((((((+.....)))))) ......(((((+......))))) {'kind': 'sliding'}


......(((((+

In [ ]:
print(G.number_of_nodes(), G.number_of_edges())
print(G1.number_of_nodes(), G1.number_of_edges())

50 73
49 48


In [ ]:
edges = list(G.edges.data())
print(*edges, sep='\n')

('...........+...........', '(((........+........)))', {'kind': 'on_nucleation'})
('...........+...........', '........(((+)))........', {'kind': 'on_nucleation'})
('...........+...........', '.....(((...+...))).....', {'kind': 'on_nucleation'})
('...........+...........', '......(((..+..)))......', {'kind': 'on_nucleation'})
('...........+...........', '....(((....+....)))....', {'kind': 'on_nucleation'})
('...........+...........', '.......(((.+.))).......', {'kind': 'on_nucleation'})
('...........+...........', '.(((.......+.......))).', {'kind': 'on_nucleation'})
('...........+...........', '..(((......+......)))..', {'kind': 'on_nucleation'})
('...........+...........', '...(((.....+.....)))...', {'kind': 'on_nucleation'})
('...........+...........', '(((........+)))........', {'kind': 'off_nucleation'})
('...........+...........', '........(((+........)))', {'kind': 'off_nucleation'})
('...........+...........', '((((.......+)))).......', {'kind': 'off_nucleation'})
('...........

In [ ]:



neighbors = nx.ego_graph(G, '...........+...........') #gets neighborhood
G.nodes[edges[1][0]] #gets node properties


{'object': <hdna.complex.Complex at 0x7efc18451670>,
 'structure': '...........+...........',
 'state': 'singlestranded',
 'pairs': 0}

In [ ]:
print(*list(neighbors.edges.data()), sep='\n')

NameError: name 'neighbors' is not defined

In [ ]:
GR[edges[1][0]].data()

AttributeError: 'AtlasView' object has no attribute 'data'

In [ ]:
simulator.biosim.species_list

OrderedCollections.OrderedDict{Symbol, BioSimulator.Species} with 50 entries:
  Symbol("...........+...........") => 0…
  Symbol("(((........+)))........") => 0…
  Symbol("((((.......+)))).......") => 0…
  Symbol("(((((......+)))))......") => 0…
  Symbol("((((((.....+)))))).....") => 0…
  Symbol("(((((((....+)))))))....") => 0…
  Symbol("((((((((...+))))))))...") => 0…
  Symbol("(((((((((..+)))))))))..") => 0…
  Symbol("((((((((((.+)))))))))).") => 0…
  Symbol(".((((((((((+.))))))))))") => 0…
  Symbol("..(((((((((+..)))))))))") => 0…
  Symbol("...((((((((+...))))))))") => 0…
  Symbol("....(((((((+....)))))))") => 0…
  Symbol(".....((((((+.....))))))") => 0…
  Symbol("......(((((+......)))))") => 0…
  Symbol(".......((((+.......))))") => 0…
  Symbol("........(((+........)))") => 0…
  Symbol("(((........+........)))") => 0…
  Symbol(".(((.......+.......))).") => 0…
  ⋮                                 => ⋮

In [ ]:
df = pd.DataFrame(list(pd.DataFrame(GR.nodes.data())[1]), index=pd.Series(GR.nodes()).index)

""" just another equivalent way of calling elements inside the dataframe """
# for i in range(len(df)):
#     print(df.iloc[i])
#     print('\n')

for i in df.index:
    print(df.loc[i])
    print('free energy:',df.loc[i]['object'].G)
    print('pairs:      ',df.loc[i]['pairs'])
    print('\n')

object       <hdna.complex.Complex object at 0x7f96b6a45e50>
structure                            ...........+...........
state                                         singlestranded
pairs                                                      0
Name: ...........+..........., dtype: object
free energy: 0
pairs:       0


object       <hdna.complex.Complex object at 0x7f96fc194bb0>
structure                            (((........+)))........
state                                           off_register
pairs                                                      3
Name: (((........+)))........, dtype: object
free energy: -3.714681142932636
pairs:       3


object       <hdna.complex.Complex object at 0x7f96b6a452e0>
structure                            ((((.......+)))).......
state                                           off_register
pairs                                                      4
Name: ((((.......+))))......., dtype: object
free energy: -4.9594014395632975
pairs:       4


ob

In [ ]:
for node in GR.nodes():
    print(node)
len(GR.nodes)

...........+...........
(((........+)))........
((((.......+)))).......
(((((......+)))))......
((((((.....+)))))).....
(((((((....+)))))))....
((((((((...+))))))))...
(((((((((..+)))))))))..
((((((((((.+)))))))))).
.((((((((((+.))))))))))
..(((((((((+..)))))))))
...((((((((+...))))))))
....(((((((+....)))))))
.....((((((+.....))))))
......(((((+......)))))
.......((((+.......))))
........(((+........)))
(((........+........)))
.(((.......+.......))).
..(((......+......)))..
...(((.....+.....)))...
....(((....+....)))....
.....(((...+...))).....
......(((..+..)))......
.......(((.+.))).......
........(((+)))........
((((.......+.......))))
(((((......+......)))))
((((((.....+.....))))))
(((((((....+....)))))))
((((((((...+...))))))))
(((((((((..+..)))))))))
((((((((((.+.))))))))))
.(((((.....+.....))))).
..(((((....+....)))))..
.(((((((...+...))))))).
...(((((...+...)))))...
..(((((((..+..)))))))..
.(((((((((.+.))))))))).
....(((((..+..)))))....
...(((((((.+.)))))))...
..(((((((((+))))

50

In [ ]:
import time

In [ ]:
def numpytranslate(s):
    table = str.maketrans({'.':'o', '(':'b', ')':'d', '+':'A'})
    return s.translate(table)

In [ ]:
string = '...(((..((..+..))..)))...'
numpytranslate(string)

'ooobbboobbooAooddoodddooo'